# Wort-Einbettungen

Wort-Einbettungen sind große Lookup-Tabellen, die jedem Wort aus einem festen Vokabular (je größer, desto besser) einen hoch-dimensionalen Vektor zuordnen (typisch sind Dimensionen zwischen 70 und 300), sodass ähnliche Worte auf ähnliche Vektoren abgebildet werden.

Zur Veranschaulichung spielen wir ein bisschen mit den Wort-Einbettungen, die von der NLP-Bibliothek [spaCy](https://spacy.io) im deutschen Sprachmodell bereitgestellt werden.

## spaCy-Sprachmodell mit Wort-Einbettungen laden

Dazu müssen wir erst per Kommando-Zeile das deutsche Sprachmodell mit spaCy aus dem Netz herunterladen:

In [ ]:
!python -m spacy download de_core_news_md

spaCy bietet drei deutsche Sprachmodelle verschiedener Größe an: klein, mittel und groß.
Um die Wort-Einbettungen verwenden zu können, laden wir das Sprachmodell in Python wie folgt:

In [ ]:
import spacy

nlp = spacy.load('de_core_news_md')
vocab = nlp.vocab

Nun erhalten wir zu jedem Wort des Vokabulars wie folgt den Wort-Vektor des Sprachmodells:

In [ ]:
vector = vocab.get_vector('Hund')
vector.shape, vector

Die Wort-Vektoren haben hier also die Dimension 300.

## Ähnlichkeit von Wort-Vektoren

Mit diesen Wort-Vektoren können wir bereits dem Computer Spiele wie "Find the odd one out" beibringen. Dabei ist die Aufgabe, aus einer Reihe von Wörtern das Wort herauszufinden, das am wenigsten zu den anderen passt.

Als ersten Schritt berechnen wir zu einer Liste von Wörtern $w_1, \ldots, w_n$ die Ähnlichkeitsmatrix der zugehörigen Wortvektoren $v_1, \ldots, v_n$. Der Eintrag der Matrix an der Stelle $i,j$ ist das Skalarprodukt von $v_i$ und $v_j$ geteilt durch die Normen der Vektoren.

In [ ]:
import numpy as np
import pandas as pd

def similarity(words):
    vectors = np.asarray(list(map(vocab.get_vector, words)))
    norms = np.linalg.norm(vectors, axis=-1)
    normed_vectors = vectors / np.reshape(norms, (-1,1))
    matrix = np.matmul(normed_vectors, normed_vectors.transpose())
    return pd.DataFrame(matrix, columns=words, index=words)

words = ['Hund', 'Katze', 'Schuh']
similarity(words)


Zur Veranschaulichung plotten wir eine Heatmap:

In [ ]:
import seaborn as sns
%matplotlib inline

words = ['Hund', 'Katze', 'Schuh', 'Wal', 'Fisch']
sns.heatmap(similarity(words))

## Spiel "Find the odd one out"

Nun implementieren wir das oben genannte Spiel und suchen aus einer Liste von Wörtern das heraus, das den anderen am unähnlichsten ist.
Dazu summieren wir die Ähnlichkeiten in der Ähnlichkeitsmatrix entlang jeder Zeile und wählen das Wort mit der niedrigsten Summe.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

def find_the_odd_one(words):
    similarity_matrix = similarity(words).values
    similarity_sums = np.sum(similarity_matrix, axis=-1)
    return words[np.argmin(similarity_sums)]

                       
find_the_odd_one(words)

Das funktioniert ganz gut:

In [ ]:
find_the_odd_one(['Computer', 'Technik', 'Informatik', 'Kochen', 'Programmieren'])

## Projektionen von Wort-Einbettungen

Wort-Einbettungen bewahren (idealerweise) nicht nur die Ähnlichkeit von Wörtern. Das kann man durch zwei-dimensionale Projektionen der Wort-Vektoren veranschaulichen, die man durch klassische Dimensionsreduktions-Verfahren wie PCA erhält.

In [ ]:
from sklearn import decomposition

def pca(words):
    vectors = np.array(list(map(vocab.get_vector, words)))
    vectors_2d = decomposition.PCA(2).fit_transform(vectors)
    return pd.DataFrame(vectors_2d, columns=['x', 'y'], index=words)
    
pca(words)

In [ ]:
import altair as alt

def project(words):
    df = pca(words).reset_index().rename(columns={'index':'Wort'})
    base = alt.Chart(df).encode(x='x', y='y')
    return base.mark_point() + base.encode(text='Wort').mark_text(dy=10)

In [ ]:
project(words)

In [ ]:
project(['Paris', 'Frankreich', 'London', 'Großbritannien', 'Berlin', 'Deutschland'])